# Detección de burbujas

**Nota Importante**: Debe mover la carpeta `ImageFromCamera` dentro de la carpeta yolov9, debido a que evitamos errores con las rutas.

## Importar paquetes

Nos movemos a la carpeta yolov9 usando los comando de consola que admiten la celdas de códigos.

In [1]:
# we move to the directory where we have yolo
%cd yolov9/
import os

HOME = os.getcwd()
print(HOME)

c:\Users\gol_m\OneDrive\Desktop\Magíster en Ciencias de la Ingeniería\2_PROYECTOS\Bubbles\code_yolo\para_tarde\yolov9
c:\Users\gol_m\OneDrive\Desktop\Magíster en Ciencias de la Ingeniería\2_PROYECTOS\Bubbles\code_yolo\para_tarde\yolov9


* La idea es que nos ubiquemos en yolov9, esto nos permitirá importar los módulos que están en esa carpeta.

In [2]:
import numpy as np
import torch
import cv2
import supervision as sv
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

# YOLOv9
from models.common import DetectMultiBackend, AutoShape
from utils.general import set_logging
from PIL import Image

# Video Demonstration
from IPython.display import HTML
from base64 import b64encode

from ultralytics import YOLO

# Cargar Modelo

Esto se realiza una vez, cargando el modelo queda en la GPU.

In [3]:
# Load a model
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = DetectMultiBackend(weights=r'best.pt', device=device, data='{dataset.location}/data.yaml', fuse=False)
model = AutoShape(model)

# configuration of some model parameters to perform inference
model.conf = 0.1

Adding AutoShape... 


## Funciones 

Se encuentran las funciones para obtener la cantidad, y diámetro

In [4]:
# Función para procesar un frame y realizar la inferencia de objetos
def procesar_frame(frame):
    # Run batched inference on a list of images
    results = model(frame)  # return a list of Results objects model(frame, conf=0.5)
    return results  # Retornamos el frame procesado

# Función para extraer estadísticas de un conjunto de frames
def features_bubble(result):
    # Area de las burbujas de una imagen
    diametro = float(result.xywh[0][:,2])
    area = float(torch.pow(diametro,2) * np.pi/4)
    # area = np.power(diametro,2)*np.pi/4
    cantidad = int(len(result.xywh[0]))
    return area, cantidad, diametro

def plot_hist(diametro):
    # Genera un vector de datos aleatorios
    datos = diametro
    # Crea el histograma
    plt.hist(datos, bins=30, density=True, alpha=0.7, color='blue')
    # Añade títulos y etiquetas
    plt.title('Histograma de burbujas')
    plt.xlabel('Diametro')
    plt.ylabel('Frecuencia')
    # Muestra el histograma
    plt.show()

# Realizar Inferencia o Post-processing

In [11]:
import os
import glob
import time
from datetime import datetime
import json

# Source and destination folder
carpeta_origen = 'ImagesFromCamera'
carpeta_destino = 'ImagePostProcessing'

# Create the folder if it does not exist
os.makedirs(carpeta_destino, exist_ok=True)

def obtener_imagen_mas_reciente(carpeta):
    """Obtiene la imagen más reciente en la carpeta especificada."""
    lista_archivos = glob.glob(os.path.join(carpeta, '*.JPG'))  # Ajusta la extensión si es necesario
    if not lista_archivos:
        return None
    imagen_mas_reciente = max(lista_archivos, key=os.path.getctime)
    return imagen_mas_reciente

def procesar_imagen(image_path, carpeta_destino):
    """Realiza la inferencia sobre la imagen y guarda el resultado."""
    results = model(image_path)

    # area, cantidad, diametro = features_bubble(results)
    diccionario = {
        "nombre archivo": image_path,
        "cantidad": len(results.xywh[0]),
        # "diametro": diametro,
        # "area": area
    }

    #debug
    # print("aquí toy")
    # print(image_path)
    nombre_archivo = os.path.basename(image_path)
    ruta_destino_para_json = os.path.join(carpeta_destino, nombre_archivo)
    nombre_archivo_json = f"{ruta_destino_para_json}.json"

    # Guardar el diccionario en un archivo JSON
    with open(nombre_archivo_json, 'w') as archivo:
        json.dump(diccionario, archivo, indent=4)

    print(f"Diccionario guardado en {nombre_archivo_json}")

    img = results.ims[0].copy()

    # Dibujar los bounding boxes en la imagen
    for box in results.xyxy[0]:  # Suponiendo que xyxy contiene las coordenadas del bounding box
        x1, y1, x2, y2, conf, cls = box.tolist()  # Convertir de tensor a lista
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # Convertir de float a int

        # Dibujar el bounding box
        color = (255, 0, 0)  # Color rojo (RGB)
        thickness = 3
        cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)

    # Guardar la imagen procesada en la carpeta destino
    nombre_archivo = os.path.basename(image_path)
    ruta_destino = os.path.join(carpeta_destino, nombre_archivo)
    cv2.imwrite(ruta_destino, img[:,:,[2,1,0]]) #invierto los canales porque queda en BGR, truquito de PDI2
    print(f'Imagen guardada en {ruta_destino}')

# Bucle para monitorizar la carpeta y procesar nuevas imágenes
ultima_imagen_procesada = None

try:
    while True:
        imagen_mas_reciente = obtener_imagen_mas_reciente(carpeta_origen)
        
        if imagen_mas_reciente != ultima_imagen_procesada:
            ultima_imagen_procesada = imagen_mas_reciente
            if imagen_mas_reciente is not None:
                procesar_imagen(imagen_mas_reciente, carpeta_destino)
        
        # Esperar un tiempo antes de volver a comprobar (por ejemplo, 5 segundos)
        time.sleep(5)
except KeyboardInterrupt:
    print("Se ha parado la ejecución del código.")


aquí toy
ImagesFromCamera\001.JPG
Diccionario guardado en ImagePostProcessing\001.JPG.json
Imagen guardada en ImagePostProcessing\001.JPG
aquí toy
ImagesFromCamera\hola.jpg
Diccionario guardado en ImagePostProcessing\hola.jpg.json
Imagen guardada en ImagePostProcessing\hola.jpg
Se ha parado la ejecución del código.
